In [ ]:
!pip install openai==1.0.0

In [1]:
import pandas as pd
jobs = pd.read_excel('/Users/taijieshengwu/Documents/专业课/大三上/boss_0.xlsx')

In [ ]:
import pandas as pd

category_mapping = {
    "内容运营":["内容运营（电商）","内容运营（跨境电商）","内容运营（小说/网文/短剧）","新媒体内容运营","运营专员","运营实习生","小红书内容运营","短视频内容运营","内容运营专员","内容运营实习生","小红书内容运营负责人","品牌内容策划","IP内容运营","抖音短视频内容运营","淘内内容运营","TikTok内容运营","短视频文案编辑","用户运营","内容电商营销运营"],
    "电商运营":["电商运营专员","电商运营经理","电商运营助理","电商运营主管","跨境电商运营专员","跨境电商运营主管","跨境电商亚马逊运营专员","跨境电商运营总监","电商渠道运营专员","新媒体运营专员","淘宝运营专员","电商直播运营专员","高级市场运营专员","电商运营实习生","阿里巴巴运营专员"],
    "编辑文案":["文案编辑","医学编辑","小说编辑","短篇编辑","新媒体编辑","财经公众号编辑","SEO网站运营","短视频脚本编辑","视频剪辑/编辑","社区运营（内容运营/文案策划）","资深文案策划","项目专员（文案）","电商文案编辑策划","网络小说编辑","游戏文案编辑"],
    "产品经理":["产品经理","产品经理（芯片）","产品经理（金融硬件）","产品经理（医疗软件）","产品经理（办公系统）","资深产品经理","商品成长产品经理","推荐策略产品经理","影像产品经理"]
}

# 定义一个函数来匹配职位并归类
def categorize_job(job_title):
    for category, keywords in category_mapping.items():
        if any(keyword in job_title for keyword in keywords):
            return category
    return job_title 

# 将职位分类
jobs['job_category'] = jobs['job'].apply(categorize_job)

# 将相同类别的职位描述合并
merged_jobs = jobs.groupby('job_category').agg({
    'detail': ' '.join  # 合并同类别的职位描述
}).reset_index()

# 只保留5个类别
merged_jobs = merged_jobs[merged_jobs['job_category'].isin(category_mapping.keys())]

# 查看结果
print(merged_jobs["job_category"])

In [ ]:
# 把job合并成"内容运营"，“电商运营”，“编辑”，“产品经理”，”广告文案“5个岗位


In [2]:
merged_jobs=[]
for _, row in jobs.iterrows():
    dict={"job": row['job'], "description": row['detail']}
    merged_jobs.append(dict)
print(len(merged_jobs))

1097


In [3]:
import os
import openai

# optional; defaults to `os.environ['OPENAI_API_KEY']`
openai.api_key = "sk-yiOQSzzduEMnzrb0B51aEc9a191b4b22B08d0d1fB1480081"

# all client options can be configured just like the `OpenAI` instantiation counterpart
openai.base_url = "https://free.gpt.ge/v1/"
openai.default_headers = {"x-foo": "true"}

def extract_job_details(job, description):
    prompt = f"""给定以下的职位描述，请提取以下字段，并且只填写具体内容，没有的就写“无”：
    1. 专业：这个职位要求的专业背景或学历。
    2. 性格：适合这个职位的性格特点。
    3. 软件技能：职位要求掌握的软件技能。
    4. 工作内容：职位的主要职责和工作内容。
    5. 品质：适合这个职位的个人品质或能力。

    职位描述：
    "{description}"

    请以以下格式返回：
    {job}
    [专业-性格-软件技能-工作内容-品质]
    """
    
    completion = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": prompt,
        },
    ],
)
    return completion.choices[0].message.content


# 初始化空列表存储结果
results = []

# 循环处理每条职位描述
for job_info in merged_jobs:
    job_title = job_info['job']
    job_description = job_info['description']
    result = extract_job_details(job_title, job_description)

    result_lines = result.split("-")
    if len(result_lines) != 6:
        result_lines.insert(0, job_title)  # 将 job_title 插入为第一个元素
    
    # 确保结果的长度为 6，即 [job, 专业, 性格, 软件技能, 工作内容, 品质]
    while len(result_lines) < 6:
        result_lines.append("无")  # 如果某些字段缺失，填充 "无" 作为占位符

    # 将结果保存到字典
    results.append({
        "job": result_lines[0],
        "专业": result_lines[1],
        "性格": result_lines[2],
        "软件技能": result_lines[3],
        "工作内容": result_lines[4],
        "品质": result_lines[5]
    })
# 将结果转换为 DataFrame
df = pd.DataFrame(results)

# 导出为 CSV 文件
df.to_csv("job_details.csv", index=False)

print("CSV 文件已生成，路径为 job_details.csv")

CSV 文件已生成，路径为 job_details.csv
